In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [9]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [10]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [11]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [12]:
train.shape

(120, 5)

In [13]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [14]:
def input_fn(features, labels, training=True, batch_size=256):
  # Pretvori is pandas u tf DataSet
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #Uradi shufle N puta
  if training:
    dataset = dataset.shuffle(1000).repeat()
  
  return dataset.batch(batch_size) #I odredjenji batch size

In [15]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [16]:
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\TEODOR~1\\AppData\\Local\\Temp\\tmp1ggxs2mf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
#Kao u Exercise 2 morali smo da imamo funkciju koja vraca funkciju kao odgovor, to smo u 2. uradili u input function
#Dok ovde smo za input function uradili tek posle prilikom trening sa LAMBDA  izrazom, zasto je ovo ovako mora da se proveri
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True), steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\TEODOR~1\AppData\Local\Temp\tmp1ggxs2mf\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.6740794, step = 0
INFO:tensorflow:global_step/sec: 541.201
INFO:tensorflow:loss = 0.9626291, step = 100 (0.186 sec)
INFO:tensorflow:global_step/sec: 693.427
INFO:tensorflow:loss = 0.78784096, step = 200 (0.144 sec

In [18]:
#Sada treba evaluirati nas trening  i traning=False jer netreba shuffle podataka jer je samo evaluacija
classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-21T18:12:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\TEODOR~1\AppData\Local\Temp\tmp1ggxs2mf\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.20365s
INFO:tensorflow:Finished evaluation at 2021-11-21-18:12:14
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.93333334, average_loss = 0.3469302, global_step = 5000, loss = 0.3469302
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\TEODOR~1\AppData\Local\Temp\tmp1ggxs2mf\model.ckpt-5000


{'accuracy': 0.93333334,
 'average_loss': 0.3469302,
 'loss': 0.3469302,
 'global_step': 5000}

In [27]:
def input_fn(features, batch_size=256):
    # Konvertuje pandas input u dataset za tf
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

#Daj svoj proizovljan primer cveta da se uporedi sa modelom
print("Please type numeric values as prompted.")
for feature in features:
    valid = True
    while valid: 
        val = input(feature + ": ")
        if not val.isdigit(): valid = False

    predict[feature] = [float(val)]

#Posto mu mi prosledjujemo samo jedan primer, tako je napravljena ova vunkcija malopre,
#Medjutim mozemo da mu prosledimo vise od jedne stvari odjednom
predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))


Please type numeric values as prompted.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\TEODOR~1\AppData\Local\Temp\tmp1ggxs2mf\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-1.0837886,  1.0290974,  7.5611053], dtype=float32), 'probabilities': array([1.7573658e-04, 1.4537070e-03, 9.9837059e-01], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (99.8%)
